# Connecting to MySQL Using Connector/Python


First of all you need an account in a MySQL server running in your computer or remotely. To run it in your computer, as a suggestion:
- In Windows or MAc, install the XAMPP bundle (https://www.apachefriends.org/index.html). 
- In Linux install the LAMP stack (https://help.ubuntu.com/community/ApacheMySQLPHP) 
- Use phpmyadmin to
    - create a database called `sensors`
    - create a `user='meec'` with `password='00meec'` and "data" and "Structure" permission in a `database='sensors'`


Make sure you have  `mysql-connector-python` package installed and import it

In [ ]:
import traceback

try:
    import mysql.connector
except:
    !pip install mysql-connector-python
    import mysql.connector

"mysql.connector.__version__: " + mysql.connector.__version__

The `connect()` constructor creates a connection to the MySQL server and returns a `MySQLConnection` object.
(see https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html for other argument options)

In [ ]:
try:
    cnx = mysql.connector.connect(
        user='meec',
        password='00meec',
        host='127.0.0.1', # in your case replace by 'localhost'
        database=''
    )
except mysql.connector.Error as e:
    print(f"problems opening the connection to mysql: {e}")
else:
    print("ok!")
    cnx.close()

Usually, it is best to 
* do exception treatment; and  
* have a configuration file (see config.py) 
```
config = {
    'host' : 'localhost',
    'user' : 'meec',
    'password' : '00meec',
    'db' : 'sensors'
}
```
and then...

In [ ]:
from config import config as conf

try:
    cnx = mysql.connector.connect(**conf)
except mysql.connector.Error as e:
    print(f"problems opening the connection to mysql: {e.msg}")
else :
    print("ok!")
    cnx.close()

# Create a schema
Lets us create the database schema (do not forget to catch exceptions, I'm simplifying!). Further, when your app is running you don't need to keep connecting and closing your connection.

In [ ]:
sql = '''
    CREATE SCHEMA IF NOT EXISTS `sensors` DEFAULT CHARACTER SET utf8 ;
    USE `sensors` ;

    -- -----------------------------------------------------
    -- Table `sensors`.`Location`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Location` (
      `idLocation` INT NOT NULL AUTO_INCREMENT,
      `name` VARCHAR(45) NOT NULL,
      `description` VARCHAR(45) NOT NULL,
      PRIMARY KEY (`idLocation`),
      UNIQUE INDEX `name_UNIQUE` (`name` ASC))
    ENGINE = InnoDB;


    -- -----------------------------------------------------
    -- Table `sensors`.`Unit`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Unit` (
      `unit` VARCHAR(45) NOT NULL,
      `description` VARCHAR(45) NOT NULL,
      PRIMARY KEY (`unit`))
    ENGINE = InnoDB;


    -- -----------------------------------------------------
    -- Table `sensors`.`Sensor`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Sensor` (
      `idSensor` INT NOT NULL AUTO_INCREMENT,
      `idLocation` INT NOT NULL,
      `name` VARCHAR(45) NOT NULL,
      `unit` VARCHAR(45) NOT NULL,
      PRIMARY KEY (`idSensor`),
      INDEX `fk_Sensor_Location_idx` (`idLocation` ASC),
      INDEX `fk_Sensor_Units1_idx` (`unit` ASC),
      UNIQUE INDEX `uniq_loc_vs_sensor` (`idLocation` ASC, `name` ASC),
      CONSTRAINT `fk_Sensor_Location`
        FOREIGN KEY (`idLocation`)
        REFERENCES `sensors`.`Location` (`idLocation`)
        ON DELETE CASCADE
        ON UPDATE CASCADE,
      CONSTRAINT `fk_Sensor_Units1`
        FOREIGN KEY (`unit`)
        REFERENCES `sensors`.`Unit` (`unit`)
        ON DELETE CASCADE
        ON UPDATE CASCADE)
    ENGINE = InnoDB;


    -- -----------------------------------------------------
    -- Table `sensors`.`Reading`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Reading` (
      `idReading` INT NOT NULL AUTO_INCREMENT,
      `idSensor` INT NOT NULL,
      `timestamp` TIMESTAMP NOT NULL,
      `value` FLOAT NOT NULL,
      PRIMARY KEY (`idReading`),
      INDEX `fk_Reading_Sensor1_idx` (`idSensor` ASC),
      CONSTRAINT `fk_Reading_Sensor1`
        FOREIGN KEY (`idSensor`)
        REFERENCES `sensors`.`Sensor` (`idSensor`)
        ON DELETE CASCADE
        ON UPDATE CASCADE)
    ENGINE = InnoDB;


    -- -----------------------------------------------------
    -- Table `sensors`.`Alert`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Alert` (
      `idAlert` INT NOT NULL AUTO_INCREMENT,
      `idSensor` INT NOT NULL,
      `timestamp` TIMESTAMP NOT NULL,
      `description` VARCHAR(45) NOT NULL,
      `cleared` BIT NULL,
      PRIMARY KEY (`idAlert`),
      INDEX `fk_Alert_Sensor1_idx` (`idSensor` ASC),
      CONSTRAINT `fk_Alert_Sensor1`
        FOREIGN KEY (`idSensor`)
        REFERENCES `sensors`.`Sensor` (`idSensor`)
        ON DELETE CASCADE
        ON UPDATE CASCADE)
    ENGINE = InnoDB;
'''

Now, let's execute the sql command to create the schema. Note that this can be done using GUI such as phpmyadmin or MySQL Workbench.

In [ ]:
try:
    cnx = mysql.connector.connect(**conf)
    cursor = cnx.cursor()
    cursor.execute(sql)
except mysql.connector.Error as err:
    print(err)
else:
    cnx.close()

# Create (INSERT) data

The INSERT statement is used to add new rows to a table (see https://dev.mysql.com/doc/refman/8.0/en/insert.html).
The INSERT basic syntax is:
```sql
INSERT INTO table_name 
    (column1, column2, column3, ...)
VALUES 
    (value1, value2, value3, ...);
``` 

In [ ]:
cnx = mysql.connector.connect(**conf)
cursor = cnx.cursor()    

Let us insert a new location and get its `id`. The `id` is a column with the `AUTO_INCREMENT` attribute, so it is automatically generated by MySQL. The `lastrowid` attribute of the cursor object contains the `id` of the last row inserted by the cursor.
(see https://dev.mysql.com/doc/refman/8.0/en/insert.html)

In [ ]:
# prepare the sql query for the new location
sql = '''INSERT INTO `Location` 
            (`idLocation`, `name`, `description`) 
         VALUES 
            (DEFAULT, %s, %s);
        '''

data = ('Prometheus Server 2022', 'Prometheus Server @ lab. 163 / ISE /UAlg')

#execute the sql query and get the new location id
cursor.execute(sql, data)

# get the new location id
location_id = cursor.lastrowid
location_id

When you use a transactional storage engine such as InnoDB, you must commit the data after a sequence of INSERT, DELETE, and UPDATE statements (any command that changes data). If you do not call `commit()`, the changes are not saved to the database.

In [ ]:
cnx.commit()

Now let us insert a new unit. 

_(REPLACE works exactly like INSERT, except that if an old row in the table has the same value as a new row for a PRIMARY KEY or a UNIQUE index, the old row is deleted before the new row is inserted. See https://dev.mysql.com/doc/refman/8.0/en/replace.html)_

The REPALCE basic syntax is:
```sql
REPLACE INTO table_name 
    (column1, column2, column3, ...)
VALUES
    (value1, value2, value3, ...);
```


In [ ]:
# Insert a new unit (if it does not exist)
sql = '''REPLACE INTO Unit 
            (unit, description) 
        VALUES 
            ("percent", "percentage of usage"); 
        '''

# execute the sql query
cursor.execute(sql)

# commit the changes - for percistence of data
cnx.commit()

Now let us insert a new sensor and get its id

In [ ]:
# prepare the sql query for the new sensor
sql = '''INSERT INTO `Sensor` 
            (`idSensor`, `idLocation`, `name`, `unit`)
        VALUES 
            (DEFAULT, %(idLocation)s, %(name)s, %(unit)s);
        '''
data = {
        'idLocation': location_id, 
        'name' : 'cpu_sensor_01', 
        'unit' : 'percent'
       }

#execute the sql query
cursor.execute(sql, data)

# get the new sensor id
sensor_id = cursor.lastrowid
cnx.commit()

And now, get some data and store it. Here we will use the `psutil` package to get the CPU usage each 0.1 seconds and store it in the database.

In [ ]:
try:
    import psutil
except:
    !pip install psutil
    import psutil

sql = '''INSERT INTO `Reading` 
            (`idReading`, `idSensor`, `timestamp`, `value`)     
        VALUES 
            (DEFAULT, %(idSensor)s, DEFAULT, %(value)s);
        '''

for _ in range(100):
    data = {
           'idSensor' : sensor_id, 
            'value' : psutil.cpu_percent(interval=0.1)
           }
    cursor.execute(sql, data) 
    cnx.commit()
    print('.', end='')


In [ ]:
cursor.close()
cnx.close()

# Querying (SELECT) data

The SELECT is used to retrieve rows selected from one or more tables, and can include UNION statements and subqueries (see https://dev.mysql.com/doc/refman/8.0/en/select.html)

The SELECT basic syntax is:
```sql
SELECT column1, column2, ...
FROM table_name
WHERE condition;
```


In [ ]:
cnx = mysql.connector.connect(**conf)
cursor = cnx.cursor()  

Let us get the location which has the description containing the string '163'

In [ ]:
sql = '''SELECT idLocation, name, description 
            FROM Location 
            WHERE description LIKE "%163%"
       '''

# execute the sql query
cursor.execute(sql)

# fetch all of the rows from the query by using the cursor as an iterator
for (idLocation, name, description) in cursor:
  print("id: {}\n\t name: {} \n\t description: {}".format(idLocation, name, description))


Now, let us get the readings with the value between 5 and 50

In [ ]:
sql = '''SELECT idReading, idSensor, timestamp, value 
            FROM Reading 
            WHERE value BETWEEN %s and %s'''
data = (5, 50)

# execute the sql query - data is a tuple with the values to be replaced in the query
cursor.execute(sql, data)

# fetch all of the rows from the query by using the cursor as an iterator
for idReading, idSensor, timestamp, value in cursor:
  print("idReading: {}\n\t idSensor: {} \n\t time: {} \n\t value: {}".format(idReading, idSensor, timestamp, value))

To get data combined from different tables, you can use the `JOIN` clause. The `JOIN` clause is used to combine rows from two or more tables, based on a related column between them. There are different types of joins, such as:
- `INNER JOIN`: returns records that have matching values in both tables
- `LEFT JOIN`: returns all records from the left table, and the matched records from the right table
- `RIGHT JOIN`: returns all records from the right table, and the matched records from the left table
- ...

The `INNER JOIN` basic syntax is:
```sql
SELECT column1, column2, ...
FROM table1
    INNER JOIN table2
        ON table1.column_name = table2.column_name;
```


In [ ]:
sql = '''SELECT idReading, r.idSensor, timestamp, value, s.name, unit, l.name, l.description
            FROM Reading AS r
            INNER JOIN Sensor AS s 
                ON r.idSensor = s.idSensor
            INNER JOIN Location as l 
                ON s.idLocation = l.idLocation
            WHERE value BETWEEN %s and %s
            '''
data = (5, 50)

# execute the sql query - data is a tuple with the values to be replaced in the query
cursor.execute(sql, data)

for (idReading, idSensor, timestamp, value, name_sensor, unit, name_location, description) in cursor:
  print(f'''idReading: {idReading}
      idSensor: {idSensor}
      time: {timestamp}
      value: {value}
      sensor name: {name_sensor}
      unit: {unit}
      location name: {name_location}
      description: {description}''')

In [ ]:
cursor.close()
cnx.close()

# Exercise

[Go to notebook ./Exercises/MySQL/Exercises.ipynb](./Exercises/MySQL/Exercises.ipynb)